# Joint Task Post-Processing

## Libraries

In [1]:
import os
import pickle
import json
import pandas as pd
import ast

In [2]:
#am_work/coling_2025/cdcp/finetuned_models_run3/CDCP_acc_mistral-7b-instruct-v0.3-bnb-4bit

In [3]:
# nb_epochs = 0.2
# OUTPUT_DIR = "../finetuned_models_run3/CDCP_acc_mistral-7b-instruct-v0.3-bnb-4bit"

In [4]:
with open("/Utilisateurs/umushtaq/am_reasoning/cdcp/saved_models/cdcp_pipeline_Meta-Llama-3.1-8B-Instruct-bnb-4bit/cdcp_pipeline_results_5.pickle", "rb") as fh:
        
        results = pickle.load(fh)

In [5]:
results

{'grounds': ['### Output:\n{"argument_types": [\'fact\', \'value\', \'policy\']}\n{"relation_types": [(3, 2, \'reason\')]}\n',
  '### Output:\n{"argument_types": [\'value\', \'policy\', \'policy\', \'policy\', \'policy\']}\n{"relation_types": []}\n',
  '### Output:\n{"argument_types": [\'policy\', \'policy\']}\n{"relation_types": []}\n',
  '### Output:\n{"argument_types": [\'value\', \'value\']}\n{"relation_types": []}\n',
  '### Output:\n{"argument_types": [\'value\', \'value\', \'value\', \'testimony\', \'value\']}\n{"relation_types": [(5, 3, \'reason\'), (5, 4, \'reason\')]}\n',
  '### Output:\n{"argument_types": [\'testimony\', \'testimony\', \'testimony\', \'value\', \'value\', \'policy\']}\n{"relation_types": [(5, 1, \'reason\'), (5, 2, \'reason\'), (5, 3, \'reason\'), (5, 4, \'reason\')]}\n',
  '### Output:\n{"argument_types": [\'value\', \'fact\', \'testimony\', \'testimony\', \'testimony\']}\n{"relation_types": [(2, 3, \'evidence\'), (2, 4, \'evidence\'), (2, 5, \'evidence\'),

In [6]:
grounds_global = results["grounds"]

In [7]:
preds_global = results["predictions"]

In [8]:
len(grounds_global), len(preds_global)

(150, 150)

## ACC post-processing

In [14]:
grounds_global[0].split("\n")[1]

'{"argument_types": [\'fact\', \'value\', \'policy\']}'

In [16]:
grounds_acc = [json.loads(x.replace('### Output:\n', '').strip().split('\n')[0].replace("'", '"'))["argument_types"] for x in grounds_global]

In [17]:
predictions_acc = [json.loads(x.replace('### Output:\n', '').strip().split('\n')[0].replace("'", '"'))["argument_types"] for x in preds_global]

In [18]:
len(grounds_acc), len(predictions_acc)

(150, 150)

In [19]:
bad_idx = []
for i,(x,y) in enumerate(zip(grounds_acc, predictions_acc)):
    
    if len(x) != len(y):
        bad_idx.append(i)
        print(i)
        #preds[i] = harmonize_preds(x, y)

73
75


In [20]:
grounds_acc = [elem for idx, elem in enumerate(grounds_acc) if idx not in bad_idx]
predictions_acc = [elem for idx, elem in enumerate(predictions_acc) if idx not in bad_idx]

In [21]:
ATC_preds = [item for row in predictions_acc for item in row]
ATC_grounds = [item for row in grounds_acc for item in row]

In [22]:
# sanity check: 
len(ATC_preds) == len(ATC_grounds)

True

In [23]:
from sklearn.metrics import classification_report

In [24]:
print(classification_report(ATC_grounds, ATC_preds, digits=3))

              precision    recall  f1-score   support

        fact      0.694     0.656     0.675       128
      policy      0.907     0.958     0.932       143
   reference      1.000     1.000     1.000         1
   testimony      0.922     0.852     0.886       237
       value      0.852     0.884     0.868       464

    accuracy                          0.857       973
   macro avg      0.875     0.870     0.872       973
weighted avg      0.857     0.857     0.856       973



### ARIC

In [32]:
import re

In [24]:
len(grounds_global), len(preds_global)

(150, 150)

In [25]:
grounds_aric = [json.loads(json.dumps(ast.literal_eval(x.replace('### Output:\n', '').strip().split('\n')[1])))["relation_types"] for x in grounds_global]

In [27]:
grounds_aric

[[[3, 2, 'reason']],
 [],
 [],
 [],
 [[5, 3, 'reason'], [5, 4, 'reason']],
 [[5, 1, 'reason'], [5, 2, 'reason'], [5, 3, 'reason'], [5, 4, 'reason']],
 [[2, 3, 'evidence'],
  [2, 4, 'evidence'],
  [2, 5, 'evidence'],
  [1, 4, 'reason'],
  [1, 5, 'reason']],
 [[1, 2, 'reason'], [4, 3, 'reason'], [6, 3, 'reason']],
 [[4, 5, 'reason'],
  [4, 6, 'reason'],
  [6, 8, 'reason'],
  [6, 9, 'reason'],
  [6, 10, 'reason']],
 [[1, 2, 'reason'], [1, 4, 'reason']],
 [[2, 3, 'reason'],
  [2, 4, 'reason'],
  [2, 5, 'reason'],
  [6, 7, 'reason'],
  [6, 8, 'reason']],
 [[3, 4, 'reason']],
 [[1, 2, 'reason'], [1, 3, 'reason'], [5, 2, 'reason']],
 [],
 [[2, 3, 'reason'], [2, 4, 'reason']],
 [[1, 2, 'reason'], [3, 4, 'reason']],
 [[1, 2, 'reason'], [1, 3, 'reason'], [1, 4, 'reason']],
 [[1, 2, 'reason'], [1, 3, 'reason'], [1, 4, 'reason']],
 [],
 [],
 [[1, 3, 'reason'], [3, 2, 'reason'], [5, 6, 'reason']],
 [[2, 3, 'reason'], [2, 4, 'reason'], [7, 8, 'reason'], [8, 9, 'reason']],
 [[3, 4, 'reason']],
 [],
 

In [35]:
def safe_eval(s):
    try:
        return ast.literal_eval(s)
    except (SyntaxError, ValueError):
        # Fix unquoted 'reason' strings like: (3, 1, reason') → (3, 1, 'reason')
        s = re.sub(r"\((\d+),\s*(\d+),\s*([a-zA-Z_][a-zA-Z0-9_]*)'\)", r"(\1, \2, '\3')", s)
        try:
            return ast.literal_eval(s)
        except Exception as e:
            print("Still failed to parse:", s)
            return {}

In [36]:
predictions_aric = []

for x in preds_global:
    try:
        output_line = x.replace('### Output:\n', '').strip().split('\n')[1]
        parsed = safe_eval(output_line)
        if isinstance(parsed, dict) and "relation_types" in parsed:
            predictions_aric.append(parsed["relation_types"])
        else:
            predictions_aric.append([])  # Fallback for malformed inputs
    except Exception as e:
        print(f"Error parsing line: {x}\n{e}")
        predictions_aric.append([])  # Add empty entry on failure

In [37]:
len(grounds_aric), len(predictions_aric)

(150, 150)

In [38]:
def process_grounds_aric(grounds_l, nr_acs):
    
    relations_aric = []
    
    relation_dict = {(i, j): rel for i, j, rel in grounds_l}
    
    for i in range(1, nr_acs + 1):
        for j in range(1, nr_acs + 1):
            if i != j:
                relation = relation_dict.get((i, j), "NR")
                relations_aric.append((i, j, relation))
                
    return relations_aric

In [42]:
nr_acs_l = [len(elem) for elem in grounds_acc]

In [40]:
ground_relations = []

for i, j in zip(grounds_aric, nr_acs_l):
    ground_relations.append(process_grounds_aric(i, j))

In [41]:
len(ground_relations)

148

In [43]:
def process_predictions_aric(predictions_l, nr_acs):
    
    relations_aric = []
    
    relation_dict = {(i, j): rel for i, j, rel in predictions_l}
    
    for i in range(1, nr_acs + 1):
        for j in range(1, nr_acs + 1):
            if i != j:
                relation = relation_dict.get((i, j), "NR")
                relations_aric.append((i, j, relation))
                
    return relations_aric

In [44]:
prediction_relations = []

for i, j in zip(predictions_aric, nr_acs_l):
    prediction_relations.append(process_predictions_aric(i, j))

In [ ]:
len(prediction_relations)

148

In [46]:
grounds_l = [elem[2] for sl in ground_relations for elem in sl]
predictions_l = [elem[2] for sl in prediction_relations for elem in sl]

In [47]:
len(grounds_l), len(predictions_l)

(8864, 8864)

In [48]:
print(classification_report(grounds_l, predictions_l, digits=3))

              precision    recall  f1-score   support

          NR      0.982     0.989     0.986      8606
    evidence      0.000     0.000     0.000        19
      reason      0.507     0.427     0.464       239

    accuracy                          0.972      8864
   macro avg      0.497     0.472     0.483      8864
weighted avg      0.968     0.972     0.969      8864

